In [1]:
import os

In [2]:
%pwd

'c:\\D-Drive\\Bhupi\\Project\\Smoke-Detector-Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\D-Drive\\Bhupi\\Project\\Smoke-Detector-Classifier'

In [5]:
import pandas as pd
import numpy as np

# Modeling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

## Data Ingestion

In [6]:
df = pd.read_csv('data/smoke.csv')
df.head()

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


## EDA

In [7]:
# Dropping unnecessary columns
df = df.drop(['Unnamed: 0', 'UTC', 'CNT'], axis=1)

In [8]:
# Checking the shape of the data
df.shape

(62630, 13)

In [9]:
# description of variables - this shows that there are no null values in the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature[C]  62630 non-null  float64
 1   Humidity[%]     62630 non-null  float64
 2   TVOC[ppb]       62630 non-null  int64  
 3   eCO2[ppm]       62630 non-null  int64  
 4   Raw H2          62630 non-null  int64  
 5   Raw Ethanol     62630 non-null  int64  
 6   Pressure[hPa]   62630 non-null  float64
 7   PM1.0           62630 non-null  float64
 8   PM2.5           62630 non-null  float64
 9   NC0.5           62630 non-null  float64
 10  NC1.0           62630 non-null  float64
 11  NC2.5           62630 non-null  float64
 12  Fire Alarm      62630 non-null  int64  
dtypes: float64(8), int64(5)
memory usage: 6.2 MB


In [10]:
# Although there are no missing values, let's confirm it
df.isnull().sum()

Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5             0
NC1.0             0
NC2.5             0
Fire Alarm        0
dtype: int64

In [11]:
# Checking for any duplicacte records
print('Duplicates:', df.duplicated().sum())

Duplicates: 2


In [12]:
# Checking the distribution of target variable
print(df['Fire Alarm'].value_counts())
print(round(df['Fire Alarm'].value_counts(normalize=True)*100,2))

Fire Alarm
1    44757
0    17873
Name: count, dtype: int64
Fire Alarm
1    71.46
0    28.54
Name: proportion, dtype: float64


In [13]:
# Initializing X and y for training purposes
X = df.drop('Fire Alarm', axis=1)
y = df['Fire Alarm']